* Logical Steps to find all possible function call sequences triggered by a user event in Android apps *

Step 1. 
Parsing the Android Manifest File: Identifying all activities and their entry points.

Step 2. 
Parsing the UI Layout Files: Extracting all UI elements and their associated event handlers.

Step 3. 
Parsing the Java Source Code Files: Identifying all the method declaration and the subsequent method calls within
                                       
Step 4. 
Building a Call Graph: Building the Call Sequences analyzing the above set of results from (Step 1) manifest, (Step 2) layout and (Step 3) source code.

Pre-requisites:
1. Need Javalang module (third party mdoule) to parse Java Soruce code file in Step 3. 

Step 1: Parsing the Android Manifest File:

In [15]:
import os                              # Reading/writing files (We will be reading Manifest File, Layout file and Java File
import xml.etree.ElementTree as ET     # Parsing XML file as we will be dealing with manifest file and layout file
from pprint import pprint              # Prettiyfying the output instead of standard print()

# Path of the directory. Here I have used three main paths: Manifest Directory (XML File), Layouts Directory (XML files) and Source Directory (.Java files)
manifest_directory = "final/app/src/main/"                          
layouts_directory = "final/app/src/main/res/layout/"
source_directory = "final/app/src/main/java/com/example/afinal/"

#Parses the Android Manifest File to identify all 'activites'      
def parse_manifest_directory(manifest_directory):

    # Searches only manifest files (xml) and stores manifest_files
    manifest_files = [os.path.join(manifest_directory, f) for f in os.listdir(manifest_directory) if f.endswith('.xml')]  
    
    activities = []                        # Empty list to hold the name of Activites found in Manifest file                     
    
    for manifest_file in manifest_files:   # Iternates over each file
        tree = ET.parse(manifest_file)     # Parse the XML files and return ElementTree object
        root = tree.getroot()              # Get the root elment of the prased SML tree. <manifest> element is generally the root element
        for activity in root.findall(".//activity"):  # finall() looks for the 'activity' elements in the tree. We are looking for activity
            activity_name = activity.get("{http://schemas.android.com/apk/res/android}name") # Get the name of the activity (namespace is used here for retrieving name) 
            activities.append(activity_name) 
    return activities

activities = parse_manifest_directory(manifest_directory)

# I am using pprint to prettify (more readable) the output instead of plain print() 
pprint(activities) 

['.SettingsActivity', '.MainActivity']


Step 2: Parsing the UI Layout Files and extracts information about any 'onClick' evetns. Nore: For this test purpose, I have only used 'onClick' UI event, it can be extended to handle other events as well. 

In [17]:
def parse_layouts_directory(layouts_directory):

    # same as above just like searching for Manifest file. But here we will most probably come across multiple layouts files (xml Files)
    layout_files = [os.path.join(layouts_directory, f) for f in os.listdir(layouts_directory) if f.endswith('.xml')]
    
    # have used dictionary, since I need to store information in key-value pair. [Key] being the IDs and [Value] being the name of the handler/method
    handlers = {}  
    
    for layout_file in layout_files:
        
        tree = ET.parse(layout_file)
        root = tree.getroot()
       
        for element in root.iter():  # iterates over all elements in the XML tre
            onClick = element.get("{http://schemas.android.com/apk/res/android}onClick") # retrieves the value of the attribute android:onClick attribute
            if onClick:
                element_id = element.get("{http://schemas.android.com/apk/res/android}id", 'unknown') # retrieve the value of the attribute ID of android:onClick attribute, if not assign it to 'unknown'
                handlers[element_id] = onClick # add the id of attribute and value of onClick in Key-Value pair in dictionary
    return handlers

handlers = parse_layouts_directory(layouts_directory)

#Display the result
pprint(handlers)

{'@+id/btnDisplayText': 'handleText',
 '@+id/btnSettings': 'launchSettings',
 '@+id/button': 'goBack'}


Step 3: Parse the Source Code: Java Files

In [19]:
# Thank god! this module saved me. I was trying to do with so many other modules (javaparser-python) but javalang did the trick
# Since we are dealing with Java file here, javalang module does the heavy work of parsing java 
import javalang 

def parse_java_source_code_directory(source_directory):
    
    call_graph = {}    ## A container to store method names and their possible call sequeces of methods within
    source_files = [os.path.join(source_directory, f) for f in os.listdir(source_directory) if f.endswith('.java')] # look for files ending with .java
    
    for source_file in source_files:
        
        with open(source_file, "r") as source:                        # Opens each file in read mode
            tree = javalang.parse.parse(source.read())                # IMPORTANT: Parses the content of the java file and returns an Abstract Syntax Tree (AST)
            for path, node in tree:
                
                if isinstance(node, javalang.tree.MethodDeclaration): # Check if the node is a method 
                    method_name = node.name                           # retrieves the name of method
                    if method_name not in call_graph:
                        call_graph[method_name] = []                  # set the key of earlier intialized call_graph as the method name (let just say, entry method name)
                    method_calls = []                                 # an empty list to store method calls within the current method.
                    for _, child_node in node:                        #  iterates over all child nodes of the current method node
                        if isinstance(child_node, javalang.tree.MethodInvocation): # checks if the child node is a method invocation.
                            method_calls.append(child_node.member)                 # add the method name
                    call_graph[method_name].extend(method_calls)                   # adds them to the call graph under the current method name.
    return call_graph

call_graph = parse_java_source_code_directory(source_directory)

#print("Call Graph:", call_graph)
pprint(call_graph)

{'goBack': ['startActivity'],
 'handleText': ['findViewById',
                'getText',
                'toString',
                'findViewById',
                'makeText',
                'show'],
 'launchSettings': ['findViewById', 'putExtra', 'startActivity'],
 'onCreate': ['enable',
              'setContentView',
              'setOnApplyWindowInsetsListener',
              'findViewById',
              'getInsets',
              'systemBars',
              'setPadding',
              'enable',
              'setContentView',
              'setOnApplyWindowInsetsListener',
              'findViewById',
              'getInsets',
              'systemBars',
              'setPadding',
              'getIntent',
              'getStringExtra',
              'findViewById']}


Step 4: (THE MOST IMPORTANT STEP) Integrate and Build the Call Graph: OR might as well say Building the Call Sequences analyzing the above set of results from manifest, layout and source code.

In [21]:
# The following two functions are used to build and analyze call sequences based on a set of UI event handlers and a call graph of method invocation.

# @param: handlers from Step 2 : Contain the information of UI element's IDs and their event handler methods
# @param: call_graph from Step 3: Contain the information of Method Declaration and Method Invocation
def build_call_sequences(handlers, call_graph):
    
    sequences = {} # container to store the call sequences for each UI element.                              
    for ui_element, handler in handlers.items():
        sequences[ui_element] = explore_calls(handler, call_graph, [])
    return sequences

# @pram: func = Name of method
# @param: call_graph = info on method declaration and method invocation
# @param: A list representing the current path of method calls
def explore_calls(func, call_graph, path):

    if func not in call_graph: # if the given method is not in call_graph, which means it does not call any subsequent methods
        return path + [func]
    paths = []                 # if the given method is in call_graph, it initializes an empty list paths "to store all possible call sequences starting from the current method"
    for called_func in call_graph[func]: # iterates over each method that the current method calls
        paths.extend(explore_calls(called_func, call_graph, path + [func])) # extends the current path with the current method.
    return paths

# Build and print call sequences
sequences = build_call_sequences(handlers, call_graph)

#print("Call Sequences:", sequences)
pprint(sequences)

{'@+id/btnDisplayText': ['handleText',
                         'findViewById',
                         'handleText',
                         'getText',
                         'handleText',
                         'toString',
                         'handleText',
                         'findViewById',
                         'handleText',
                         'makeText',
                         'handleText',
                         'show'],
 '@+id/btnSettings': ['launchSettings',
                      'findViewById',
                      'launchSettings',
                      'putExtra',
                      'launchSettings',
                      'startActivity'],
 '@+id/button': ['goBack', 'startActivity']}
